In [31]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [32]:
import numpy as np
import time

from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split


In [33]:
X, y = load_diabetes(return_X_y=True)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [35]:
reg = LinearRegression()

reg.fit(X_train, y_train)

LinearRegression()

In [36]:
reg_coef = reg.coef_
reg_intercept = reg.intercept_

In [37]:
print(reg_coef)
print(reg_intercept)

[  -9.15865318 -205.45432163  516.69374454  340.61999905 -895.5520019
  561.22067904  153.89310954  126.73139688  861.12700152   52.42112238]
151.88331005254167


In [38]:
y_pred = reg.predict(X_test)

r2score = r2_score(y_test, y_pred)

In [39]:
print(r2score)

0.4399338661568969


# Stochastic Regressor

In [40]:
class SelfSGDRegressor:

    def __init__(self, lr = 0.01, epochs=100):
        self.lr = lr
        self.epochs = epochs
        self.coef_ = None
        self.intercept_ = None


    def fit(self, X_train, y_train):
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1]) # making coef as 1

        for i in range(self.epochs):

            for j in range(X_train.shape[0]):

                    idx = np.random.randint(0, X_train.shape[0])

                    y_hat = np.dot(X_train[idx], self.coef_) + self.intercept_
                    # y = mx + b ( for individual row)

                    slope_intercept = 2*(y_hat - y_train[idx])   # intercept_der -  dl/db = 2*(y_hat - y)
                    self.intercept_ = self.intercept_ - (self.lr * slope_intercept)

                    slope_coef = 2*np.dot((y_hat - y_train[idx]), X_train[idx])  # coef_der - dl/dm = 2*( (y_hat - y)*X )
                    self.coef_ = self.coef_ - self.lr*slope_coef


        print(self.coef_, self.intercept_)



    def predict(self, X_test):
        return np.dot(X_test, self.coef_) + self.intercept_

In [44]:
self_sgd = SelfSGDRegressor(lr = 0.01, epochs=50)

start = time.time()

self_sgd.fit(X_train, y_train)

print("The time taken is",time.time() - start)


[  56.11790562  -62.90247681  354.59945532  246.57920383   23.53993824
  -18.27988381 -178.40778618  138.08836877  317.74848227  122.07687809] 154.00585823610373
The time taken is 0.26718950271606445


In [45]:
y_pred = self_sgd.predict(X_test)

r2score_sgd = r2_score(y_test, y_pred)

print(r2score_sgd)

0.4315038229190553


# SGD Regressor from sklearn

In [47]:
sgd = SGDRegressor(max_iter=100,learning_rate='constant',eta0=0.01)

sgd.fit(X_train, y_train)


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGDRegressor(learning_rate='constant', max_iter=100)

In [48]:
y_pred = sgd.predict(X_test)
r2score = r2_score(y_test, y_pred)

print(r2score)

0.43290686044882687
